<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/text_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Key Words & Documents Clusters

In [1]:
import pandas as pd

In [2]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

Saving beer_5k.csv to beer_5k.csv


In [3]:
# Dataframe processing
df = pd.read_csv('beer_5k.csv')
texts = [item.replace("\t", " ")[:3000] for item in df['text'] [:3000]]
#df.head(2)

In [4]:
#Create a single blob of text
corpus = ' '.join(texts)

In [5]:
%%capture
!pip install yake

In [6]:
import yake

In [7]:
# YAKE Config
kw_extractor = yake.KeywordExtractor()
language = 'en'
max_ngram_size = 2
deduplication_threshold = 0.9
numOfKeywords = 50

In [8]:
#Keyword for the corpus
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(corpus)
#keywords

In [9]:
#Keyword for the individual documents
custom_kw_extractor_docs = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=10, features=None)

In [10]:
#Keywords per document
distilled_docs = []
for item in texts:
  keywords = custom_kw_extractor_docs.extract_keywords(item)
  tmp = [item[0] for item in keywords]
  distilled_docs.append(tmp)

In [11]:
ipa_class = []
for ind, item in enumerate(distilled_docs):
  if 'IPA' in item:
    ipa_class.append((ind,item))

In [12]:
ipa_class[10]

(346,
 ['Reviewed',
  'Decent IPA',
  'hops',
  'citrus',
  'bitter',
  'IPA',
  'off-white head',
  'Pours',
  'head',
  'Aroma'])

In [23]:
print(texts[346])

Reviewed on 1/3/2010. Can. Pours a clear deep orange with a thick off-white head. Aroma of piney, grassy hops, some citrus and tropical fruits. Flavor of caramel malt and light citrus upfront with a bitter hops finish. Decent IPA, rather on the bitter side.


In [12]:
other = []
for ind, item in enumerate(distilled_docs):
  if 'IPA' not in item:
    other.append((ind,item))

Dicts to be used to classification finetuning

In [13]:
texts_IPA = []
for item in ipa_class:
  texts_IPA.append(texts[item[0]])

In [14]:
dict_IPA = {'idx': [item for item in range(len(texts_IPA))], 'sentence': texts_IPA, 'label': [0]*len(texts_IPA)}

In [15]:
texts_other = []
for item in other[:170]:
  texts_other.append(texts[item[0]])

In [16]:
dict_other = {'idx': [item for item in range(len(texts_other))], 'sentence': texts_other, 'label': [1]*len(texts_other)}

In [17]:
dict_train = {'idx': [item for item in range(len(texts_other))]+[item for item in range(len(texts_IPA))], 
              'sentence': texts_other+texts_IPA, 'label': [1]*len(texts_other)+[0]*len(texts_IPA)}

In [21]:
dataset_train = Dataset.from_dict(dict_train)
dataset_train_few = dataset_train.shuffle(seed = 42).select(range(16))

In [ ]:
for i in range(10):
  print(dataset_train_few[i])

In [25]:
# Load a SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=dataset_train_few,
    eval_dataset=dataset_train,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=16,
    num_iterations=20, # The number of text pairs to generate for contrastive learning
    num_epochs=1, # The number of epochs to use for contrastive learning
    column_mapping={"sentence": "text", "label": "label"} # Map dataset columns to text/label expected by trainer
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
trainer.train()
#metrics = trainer.evaluate()

Applying column mapping to training dataset
***** Running training *****
  Num examples = 640
  Num epochs = 1
  Total optimization steps = 40
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Transformers

In [ ]:
%pip install -U sentence-transformers

In [ ]:
from transformers import pipeline
%%capture
model = SentenceTransformer('all-MiniLM-L6-v2')

SetFit Model

In [ ]:
!pip install setfit neural_compressor optimum[onnxruntime]==1.5.0 onnxruntime_extensions

In [18]:
%%capture
!pip install setfit

In [19]:
%%capture
!pip install datasets

In [20]:
from datasets import Dataset, load_dataset
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer

In [ ]:
# Load a dataset from the Hugging Face Hub
dataset = load_dataset("sst2")

# Simulate the few-shot regime by sampling 8 examples per class
num_classes = 2
train_dataset = dataset["train"].shuffle(seed=42).select(range(8 * num_classes))
eval_dataset = dataset["validation"]

In [ ]:
for i in range(16):
  print(train_dataset[i])

Scikit Learn Head

In [ ]:
# Load a SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=16,
    num_iterations=20, # The number of text pairs to generate for contrastive learning
    num_epochs=1, # The number of epochs to use for contrastive learning
    column_mapping={"sentence": "text", "label": "label"} # Map dataset columns to text/label expected by trainer
)

In [31]:
# Train and evaluate
trainer.train()
#metrics = trainer.evaluate()

Applying column mapping to training dataset
***** Running training *****
  Num examples = 640
  Num epochs = 1
  Total optimization steps = 40
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
metrics = trainer.evaluate()

In [39]:
metrics

{'accuracy': 0.6559633027522935}

In [36]:
# Push model to the Hub
#trainer.push_to_hub("my-awesome-setfit-model")

# Download from Hub and run inference
#model = SetFitModel.from_pretrained("lewtun/my-awesome-setfit-model")
# Run inference
preds = trainer.model.predict(["Life sucks!", "pineapple on pizza is the worst 🤮"])

In [37]:
preds

array([0, 0])

# Useful Code Snippets

TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
dataset = [
    "I enjoy reading about Machine Learning and Machine Learning is my PhD subject",
    "I would enjoy a walk in the park",
    "I was reading in the library"
]

In [ ]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(dataset)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head(25))

spaCY

In [ ]:
import spacy 
from spacy.language import Language 

In [ ]:
# text processing      
nlp = spacy.load('en_core_web_sm')        
#df['Tokens'] = df.Sumcription.apply(lambda x: nlp(x))    
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS        
#spacy_stopwords.add('attach')
#df['Lema_Token']  = df.Tokens.apply(lambda x: " ".join([token.lemma_ for token in x if token not in spacy_stopwords]))

In [ ]:
@Language.component("info_component")
def my_component(doc):
    #print(f"After tokenization, this doc has {len(doc)} tokens.")
    print("The lemmatized tokens are:", [token.lemma_ for token in doc])
    return doc

nlp = spacy.load("en_core_web_sm", disable = ['ner'])
pipeline = ["tok2vec", "parser", "lemmatizer"]

nlp.add_pipe("info_component", name="print_info",last=True)
print(nlp.pipe_names)  # ['tagger', 'parser', 'ner', 'print_info']
doc = nlp("This is a sentence.")

In [ ]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
tokens = [token.lemma_ for token in doc if token.lemma_ not in spacy_stopwords and not token.is_punct]  

In [ ]:
#tensors
vectors = [item.tensor for item in doc]
# not lemmatized tokens
tokens = [item.text for item in doc]

In [ ]:
# NLP Pipelien
nlp = spacy.load("en_core_web_sm")
doc = nlp("This is a sentence.")
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [ ]:
for doc in nlp.pipe(texts[:20]):
  print(doc.ents)

In [ ]:
texts_spacy = []
for doc in nlp.pipe(texts[:3000]):
  tokens = [token.lemma_ for token in doc if token.lemma_ not in spacy_stopwords and not token.is_punct]

  texts_spacy.append(' '.join(tokens))

In [ ]:
texts_spacy_texts = []
for doc in nlp.pipe(texts[:3000]):
  tokens = [token.text for token in doc if token.text not in spacy_stopwords and not token.is_punct]
  texts_spacy_texts.append(' '.join(tokens))

In [ ]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(texts[:3000])
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head(50))

Rake-NLTK

In [ ]:
%%capture
!pip install rake_nltk

In [ ]:
from rake_nltk import Rake
import nltk
%%capture
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
rake_nltk_var = Rake(include_repeated_phrases=False, min_length=1, max_length=3)
rake_nltk_var.extract_keywords_from_text(long_text)
keyword_extracted = rake_nltk_var.get_ranked_phrases_with_scores()
#print(keyword_extracted)

Higging Face SetFit training block (alternative to the above)

In [ ]:
# Load a SetFit model from Hub
model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
    use_differentiable_head=True,
    head_params={"out_features": num_classes},
)

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=16,
    num_iterations=20, # The number of text pairs to generate for contrastive learning
    num_epochs=1, # The number of epochs to use for contrastive learning
    column_mapping={"sentence": "text", "label": "label"} # Map dataset columns to text/label expected by trainer
)

# Train and evaluate
trainer.freeze() # Freeze the head
trainer.train() # Train only the body

# Unfreeze the head and freeze the body -> head-only training
trainer.unfreeze(keep_body_frozen=True)
# or
# Unfreeze the head and unfreeze the body -> end-to-end training
trainer.unfreeze(keep_body_frozen=False)

trainer.train(
    num_epochs=25, # The number of epochs to train the head or the whole model (body and head)
    batch_size=16,
    body_learning_rate=1e-5, # The body's learning rate
    learning_rate=1e-2, # The head's learning rate
    l2_weight=0.0, # Weight decay on **both** the body and head. If `None`, will use 0.01.
)
metrics = trainer.evaluate()

# Push model to the Hub
trainer.push_to_hub("my-awesome-setfit-model")

# Download from Hub and run inference
model = SetFitModel.from_pretrained("lewtun/my-awesome-setfit-model")
# Run inference
preds = model(["i loved the spiderman movie!", "pineapple on pizza is the worst 🤮"]) 